In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
from tqdm import tqdm
import time
import json
import os
import gc

In [2]:
import requests

cookies = {
    '2A21_92c8_saltkey': 'S1mX69z6',
    '2A21_92c8_lastvisit': '1684768064',
    '__utmz': '30105572.1684772353.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none)',
    '__utma': '30105572.135848573.1684772353.1684772353.1684850662.2',
    '__utmc': '30105572',
    '__utmt': '1',
    '2A21_92c8_sid': '66Ah1z',
    '2A21_92c8_viewid': 'tid_524607',
    '2A21_92c8_visitedfid': '715D369D889D341',
    '2A21_92c8_forum_lastvisit': 'D_905_1684772352D_457_1684850685D_341_1684850689D_889_1684850710D_369_1684850714D_715_1684851023',
    '2A21_92c8_lastact': '1684851023%09home.php%09misc',
    '2A21_92c8_sendmail': '1',
    '__utmb': '30105572.10.10.1684850662',
}

headers = {
    'authority': 'www.jbtalks.cc',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
    'cache-control': 'no-cache',
    # 'cookie': '2A21_92c8_saltkey=S1mX69z6; 2A21_92c8_lastvisit=1684768064; __utmz=30105572.1684772353.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __utma=30105572.135848573.1684772353.1684772353.1684850662.2; __utmc=30105572; __utmt=1; 2A21_92c8_sid=66Ah1z; 2A21_92c8_viewid=tid_524607; 2A21_92c8_visitedfid=715D369D889D341; 2A21_92c8_forum_lastvisit=D_905_1684772352D_457_1684850685D_341_1684850689D_889_1684850710D_369_1684850714D_715_1684851023; 2A21_92c8_lastact=1684851023%09home.php%09misc; 2A21_92c8_sendmail=1; __utmb=30105572.10.10.1684850662',
    'pragma': 'no-cache',
    'referer': 'https://www.jbtalks.cc/forum-905-1.html',
    'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
}

# response = requests.get('https://www.jbtalks.cc/forum.php', cookies=cookies, headers=headers)

In [3]:
files = glob('threads/*.json')
len(files)

574

In [10]:
urls = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)

    for d in data:
        if d[-1] > 1:
            url = d[0]
            for i in range(2, d[-1] + 1, 1):
                splitted = url.split('-')
                splitted[-2] = str(i)
                urls.append('-'.join(splitted))

100%|████████████████████████████████████████| 574/574 [00:00<00:00, 988.76it/s]


In [14]:
urls = sorted(list(set(urls)))
len(urls)

116762

In [15]:
def get_url(url):
    while True:
        try:
            response = requests.get(url, cookies=cookies, headers=headers)
            soup = BeautifulSoup(response.content, "lxml")
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)
    
    td = soup.find_all('td', {'class': 't_f'})
    td = [str(td_) for td_ in td]
    return td

In [17]:
get_url(urls[1])

['<td class="t_f" id="postmessage_1192683963">\n<div class="quote"><blockquote>那么凶的乌鸦==<br/>\r\nbb弹的shiper也打不下乌鸦的，对他们没什么杀伤力<br/>\r\n有一种打喇叭弹的就可以<br/>\r\n不 ...<br/>\n<font size="2"><font color="#999999">leowyanwei 发表于 2012-8-23 05:41 PM</font> <a href="http://www.jbtalks.cc/redirect.php?goto=findpost&amp;pid=1192683911&amp;ptid=1002534" target="_blank"><img alt="" border="0" class="zoom" id="aimg_yDnfX" onclick="zoom(this, this.src, 0, 0, 0)" onload="thumbImg(this)" onmouseover="img_onmouseoverfunc(this)" src="http://www.jbtalks.cc/images/common/back.gif"/></a></font></blockquote></div><br/>\n<br/>\n这产品不能引进，稍微修改就是真枪了。尤其是弹药，已经不是BB蛋了，而起钢弹了。</td>',
 '<td class="t_f" id="postmessage_1192684000">\n<div class="quote"><blockquote>这产品不能引进，稍微修改就是真枪了。尤其是弹药，已经不是BB蛋了，而起钢弹了。<br/>\n<font size="2"><font color="#999999">kelvin4896 发表于 2012-8-23 05:44 PM</font> <a href="http://www.jbtalks.cc/redirect.php?goto=findpost&amp;pid=1192683963&amp;ptid=1002534" target="_blank"><img alt="" border="0" class="zoom

In [18]:
!mkdir pages

In [19]:
max_worker = 30
for i in tqdm(range(0, len(urls), max_worker)):
    
    filename = os.path.join('pages', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    gc.collect()
    
    results = []
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_url, url): url for url in urls[i: i + max_worker]}

        for future in as_completed(futures):
            results.append(future.result())
    
    with open(filename, 'w') as fopen:
        json.dump(results, fopen)
        
    try:
        del results
        del futures
    except:
        pass

 36%|█████████████▎                       | 1407/3893 [53:38<1:30:36,  2.19s/it]encoding error : input conversion failed due to input error, bytes 0xF8 0xC9 0xA2 0xBB
encoding error : input conversion failed due to input error, bytes 0xF8 0xC9 0xA2 0xBB
encoding error : input conversion failed due to input error, bytes 0xF8 0xC9 0xA2 0xBB
 45%|███████████████▊                   | 1756/3893 [1:06:33<1:56:31,  3.27s/it]encoding error : input conversion failed due to input error, bytes 0xED 0x33 0xBC 0xFE
encoding error : input conversion failed due to input error, bytes 0xED 0x33 0xBC 0xFE
encoding error : input conversion failed due to input error, bytes 0xED 0x33 0xBC 0xFE
 79%|█████████████████████████████▎       | 3088/3893 [2:01:59<28:24,  2.12s/it]encoding error : input conversion failed due to input error, bytes 0xAE 0xBF 0xDE 0xB9
encoding error : input conversion failed due to input error, bytes 0xAE 0xBF 0xDE 0xB9
encoding error : input conversion failed due to input error, byt

In [20]:
files = glob('threads/*.json')
len(files)

574

In [23]:
texts = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for d in data:
        for d_ in d[1]:
            texts.append(BeautifulSoup(d_, "lxml").text)

100%|█████████████████████████████████████████| 574/574 [00:15<00:00, 37.01it/s]


In [24]:
files = glob('pages/*.json')
len(files)

3893

In [25]:
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for d in data:
        for d_ in d:
            texts.append(BeautifulSoup(d_, "lxml").text)

100%|███████████████████████████████████████| 3893/3893 [03:21<00:00, 19.28it/s]


In [26]:
texts = list(set(texts))
len(texts)

802954

In [27]:
with open('everything.jsonl', 'w') as fopen:
    for t in tqdm(texts):
        if len(t):
            fopen.write(f'{json.dumps(t)}\n')

100%|███████████████████████████████| 802954/802954 [00:01<00:00, 557520.61it/s]
